In [ ]:
import pandas as pd
import re

In [ ]:
# Import data
df = pd.read_csv("data/State_of_data_BR_2023_Kaggle - df_survey_2023.csv")

In [ ]:
# mudando o nome das colunas

column_dict = {}

for col in df.columns:

    match = re.match(r"\('(.+?)', '(.+?)'\)", col)
    if match:
        cod, pergunta = match.groups()
        column_dict[cod.replace(" ", "")] = pergunta
    else:
        print(col)

df.columns = column_dict.keys()

In [ ]:
# Dataprep - toda transformação deve ser criada sobre o teste e somente aplicada sobre o treino
